# Research Agent with Multi-Tool Orchestration

In [ ]:
from kamiwaza_sdk import KamiwazaClient as kz
import openai
from pydantic import BaseModel, Field
from typing import Dict, Any, List, Optional
import json
import time
import requests
import uuid


In [ ]:
# Initialize Kamiwaza client
client = kz("http://localhost:7777/api/")
print("Active deployments:", client.serving.list_active_deployments())

# Get OpenAI-compatible client
openai_client = client.openai.get_client('Qwen3-8B-GGUF')

# Brave Search API key
BRAVE_API_KEY = "BSAGjYS3hVm2VweZ_D8iG4VPt1Xzj5D"

In [ ]:
# Tool parameter schemas
class WebSearchParams(BaseModel):
    query: str = Field(..., description="Search query to find information on the web")

class CreateReportParams(BaseModel):
    title: str = Field(..., description="Title of the research report")
    sections: Dict[str, str] = Field(..., description="Dictionary mapping section names to content")

# Brave search implementation
def web_search(query: str) -> Dict[str, Any]:
    """Search the web using Brave Search API"""
    print(f"\n🔍 Searching for: '{query}'")
    
    url = "https://api.search.brave.com/res/v1/web/search"
    headers = {
        "Accept": "application/json",
        "X-Subscription-Token": BRAVE_API_KEY
    }
    params = {
        "q": query,
        "count": 5  # Get top 5 results
    }
    
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        
        # Extract relevant information from results
        results = []
        for item in data.get("web", {}).get("results", [])[:5]:
            results.append({
                "title": item.get("title", ""),
                "url": item.get("url", ""),
                "description": item.get("description", ""),
                "age": item.get("age", "")
            })
        
        return {
            "query": query,
            "results": results,
            "status": "success"
        }
    except Exception as e:
        return {
            "query": query,
            "results": [],
            "status": "error",
            "error": str(e)
        }

# Report creation implementation
def create_report(title: str, sections: Dict[str, str]) -> Dict[str, Any]:
    """Create a formatted research report"""
    print(f"\n📝 Creating report: '{title}'")
    
    # Format the report
    report = f"# {title}\n\n"
    report += f"*Generated on: {time.strftime('%Y-%m-%d %H:%M:%S')}*\n\n"
    
    for section_name, content in sections.items():
        report += f"## {section_name}\n\n{content}\n\n"
    
    # Save to a file (optional)
    filename = f"research_report_{int(time.time())}.md"
    with open(filename, 'w') as f:
        f.write(report)
    
    return {
        "status": "success",
        "filename": filename,
        "preview": report[:500] + "..." if len(report) > 500 else report
    }

# Tool schemas for OpenAI SDK
tool_schemas = [
    {
        "type": "function",
        "function": {
            "name": "web_search",
            "description": "Search the web for information on any topic",
            "parameters": WebSearchParams.schema()
        }
    },
    {
        "type": "function",
        "function": {
            "name": "create_report",
            "description": "Create a formatted research report with sections",
            "parameters": CreateReportParams.schema()
        }
    }
]

# Tool registry
tool_registry = {
    "web_search": web_search,
    "create_report": create_report
}

print("✅ Research tools registered and ready!")


## Research Agent System Prompt

In [ ]:
RESEARCH_AGENT_PROMPT = """You are an expert research assistant with access to web search and report creation tools. Your goal is to conduct thorough, balanced research on any topic given to you.

## Your Research Process:
1. **Plan**: Break down the research topic into key aspects to investigate
2. **Search**: Use web_search to gather information from multiple angles
3. **Analyze**: Extract key findings, identify patterns, and note different viewpoints
4. **Synthesize**: Organize your findings into a coherent, balanced report

## Guidelines:
- Always search for multiple perspectives (pros/cons, different viewpoints, recent developments)
- Make 3-5 targeted searches to gather comprehensive information
- Look for credible sources and recent information
- Present balanced findings, acknowledging different viewpoints
- Use the create_report tool to produce a well-structured final report

## Report Structure:
Your reports should typically include:
- Executive Summary: Key findings in brief
- Background/Context: Why this topic matters
- Main Findings: Organized by themes or aspects
- Different Perspectives: Contrasting viewpoints if applicable
- Conclusion: Balanced summary of the research

Remember: You're not advocating for any position - you're gathering and presenting information objectively."""



## Execute Research Task

In [ ]:
def run_research_agent(research_topic: str, max_iterations: int = 2):
    """Run the research agent with a given topic"""
    
    messages = [
        {"role": "system", "content": RESEARCH_AGENT_PROMPT},
        {"role": "user", "content": f"Please research the following topic: {research_topic}"}
    ]
    
    print(f"\n🤖 Research Agent starting research on: '{research_topic}'\n")
    print("=" * 80)
    
    iteration = 0
    while iteration < max_iterations:
        iteration += 1
        print(f"\n--- Iteration {iteration} ---")
        
        # Get model response
        response = openai_client.chat.completions.create(
            model="model",
            messages=messages,
            tools=tool_schemas,
            tool_choice="auto",
            stream=False  # For simplicity, not streaming in this example
        )
        
        choice = response.choices[0]
        assistant_message = choice.message
        
        # Add assistant's response to conversation
        messages.append({
            "role": "assistant",
            "content": assistant_message.content,
            "tool_calls": assistant_message.tool_calls if assistant_message.tool_calls else None
        })
        
        # Print any reasoning/content
        if assistant_message.content:
            print(f"\n💭 Agent thinking: {assistant_message.content[:200]}...")
        
        # Check if we're done (no tool calls)
        if choice.finish_reason == "stop" and not assistant_message.tool_calls:
            print("\n✅ Research complete!")
            break
        
        # Execute tool calls
        if assistant_message.tool_calls:
            for tool_call in assistant_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                
                print(f"\n🛠️  Executing: {function_name}")
                print(f"   Args: {function_args}")
                
                # Execute the tool
                tool_response = tool_registry[function_name](**function_args)
                
                # Add tool response to conversation
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": json.dumps(tool_response)
                })
                
                # Show abbreviated response
                if function_name == "web_search":
                    print(f"   Found {len(tool_response.get('results', []))} results")
                elif function_name == "create_report":
                    print(f"   Report created: {tool_response.get('filename', 'unknown')}")
    
    print("\n" + "=" * 80)
    print("🏁 Research agent task completed!")
    
    return messages

In [ ]:
# Example usage
research_topic = "ideal customer profile of BlueAlly"
conversation_history = run_research_agent(research_topic)